In [1]:
import collections

import numpy as np
from chainer.datasets import TupleDataset


class DatsetParseError(Exception):
    pass


def _parse_single(line):
    data = line.strip().split()[:48]
    rank = int(data[0])
    qid = int(data[1][4:])
    val = map(lambda x: float(x.split(':')[1]), data[2:])
    return qid, rank, val


def create_dataset(path, size=-1):
    """
    Create dataset from MQ2008 data.
    .. warning:: It will create dataset with label in range [0, 1, 2]
        It should be no problem for Permutation Probability Loss
        but do not plug in other loss function.
    """
    data = collections.defaultdict(lambda: [[], []])
    with open(path, mode='r') as fin:
        # Data has one json per line
        for i, line in enumerate(fin):
            q, r, v = _parse_single(line)
            if r not in {0, 1, 2}:
                raise DatasetParseError(
                    "L%d: Score must be 0, 1 or 2, but found %d" %
                    (i, r)
                )
            data[q][0].append(r)
            data[q][1].append(v)
    vectors = []
    scores = []
    for d in data.values():
        v = np.array(d[1], dtype=np.float32)
        s = np.array(d[0], dtype=np.float32)
        vectors.append(v)
        scores.append(s)
    s = max(map(len, scores))
    vectors_pad = np.zeros((len(vectors), s, v.shape[-1]), dtype=np.float32)
    scores_pad = np.zeros((len(scores), s), dtype=np.float32)
    length = np.empty((len(scores)), dtype=np.int32)
    for i, (s, v) in enumerate(zip(scores, vectors)):
        vectors_pad[i, :len(v), :] = v
        scores_pad[i, :len(s)] = s
        length[i] = len(v)

    if size > 0:
        # Sample data AFTER all data has been loaded. This is because
        # There might be bias in data ordering.
        ind = np.random.permutation(len(vectors))[:size]
        return TupleDataset(vectors_pad[ind], scores_pad[ind], length[ind])
    else:
        return TupleDataset(vectors_pad, scores_pad, length)

In [2]:
from __future__ import absolute_import, division, print_function

import chainer
import chainer.functions as F
import chainer.links as L
import chainer.initializers as I
import numpy as np


class ListNet(chainer.Chain):
    def __init__(self, input_size, n_units, dropout):
        super(ListNet, self).__init__(
            l1=L.Linear(input_size, n_units, initialW=I.GlorotUniform()),
            l2=L.Linear(n_units, n_units, initialW=I.GlorotUniform()),
            l3=L.Linear(n_units, 1, initialW=I.GlorotUniform(),
                        nobias=True)
        )
        self.add_persistent("_dropout", dropout)

    def __call__(self, x, train=True):
        s = list(x.shape)
        n_tokens = np.prod(s[:-1])
        x = F.reshape(x, (n_tokens, -1))
        if self._dropout > 0.:
            x = F.dropout(x, self._dropout, train=train)
        o_1 = F.relu(self.l1(x))

        if self._dropout > 0.:
            o_1 = F.dropout(o_1, self._dropout, train=train)
        o_2 = F.relu(self.l2(o_1))

        if self._dropout > 0.:
            o_2 = F.dropout(o_2, self._dropout, train=train)
        o_3 = self.l3(o_2)

        return F.reshape(o_3, s[:-1])

In [3]:
import logging

import chainer
import chainer.functions as F
import numpy as np


def mean_average_precision(probs, labels, length, th):
    """
    Calculate mean average precision.
    Label 1 and 2 are regarded "correct" as in the original evaluation
    script https://onedrive.live.com/?authkey=%21ACnoZZSZVfHPJd0&cid=8FEADC23D838BDA8&id=8FEADC23D838BDA8%21122&parId=8FEADC23D838BDA8%21107&o=OneUp
    Args:
        probs (numpy.ndarray): list of lists of probability predictions.
            [[0.1, 0.8, 0.3, 0.1, 0.6, 0.6, 0.2], [0., ...]...]
        labels (numpy.ndarray): list of lists of ground-truth labels.
            Each value must be 0, 1 or 2
        order (str): {'descending' or 'accending'}
    Return:
        float
    """
    num_queries = len(probs)
    out = 0.0

    for i in xrange(len(probs)):
        r = probs[i][:length[i]].argsort()
        r = r[::-1]
        candidates = labels[i, r]
        avg_prec = 0.
        precisions = []
        num_correct = 0.
        for i in xrange(min(th, len(candidates))):
            if candidates[i] >= 1:
                num_correct += 1
                precisions.append(num_correct / (i + 1))

        if precisions:
            avg_prec = sum(precisions) / len(precisions)

            out += avg_prec
    return out / float(num_queries)


def logsumexp(x, mask, zero_pad, axis):
    x_exp = F.where(mask, F.exp(x), zero_pad)
    return F.log(F.sum(x_exp, axis=axis))


def logsoftmax_no_mask(x, mask, zero_pad, axis):
    #x = x - F.broadcast_to(F.max(x, keepdims=True), x.shape)
    x_logsumexp = logsumexp(x, mask, zero_pad, axis)

    # log_p: (batch size, n)
    return x - F.broadcast_to(F.expand_dims(x_logsumexp, 1), x.shape)


def logsoftmax(x, mask, zero_pad, axis):
    return F.where(mask, logsoftmax_no_mask(x, mask, zero_pad, axis), zero_pad)


def softmax(x, mask, zero_pad, axis):
    x_explogsoftmax = F.exp(logsoftmax_no_mask(x, mask, zero_pad, axis))
    return F.where(mask, x_explogsoftmax, zero_pad)



def permutation_probability_loss(x, t, length):
    """Calculate permutation probability distributions (k=1) and measure the
    cross entropy over the two distributions.
    Args:
        x (Variable): Variable holding a 2d array whose element
            indicates unnormalized log probability: the first axis of the
            variable represents the number of samples, and the second axis
            represents the number data in a query.
        t (Variable): Variable holding a 2d float32 vector of ground truth
            scores. Must be in same size as x.
    Returns:
        Variable: A variable holding a scalar array of the loss.
    """
    length = length.reshape(-1, 1)
    mask = np.tile(np.arange(x.shape[1]).reshape(1, -1), (x.shape[0],  1)) < length
    mask = chainer.Variable(mask)
    padding = chainer.Variable(np.zeros(x.shape, dtype=x.dtype))

    # log_p: (batch size, n)
    log_p_x = logsoftmax(x, mask, padding, axis=1)
    # p_t: (batch size, n)
    log_p_t = logsoftmax(t, mask, padding, axis=1)

    # loss normalized over all instances
    loss = F.exp(log_p_t) * log_p_t - F.exp(log_p_t) * log_p_x

    return F.sum(loss) / float(x.shape[0])


def clip_data(x, l):
    return x[:, :max(l)]


def _run_batch(model, optimizer, batch, device, train):
    assert train == (optimizer is not None)
    model.cleargrads()

    x, t, l = chainer.dataset.concat_examples(batch, device=device)
    x = clip_data(x, l)
    t = clip_data(t, l)

    y = model(chainer.Variable(x), train=train)
    loss = permutation_probability_loss(y, chainer.Variable(t), l)
    acc = mean_average_precision(y.data, t, l, 100000)
    if optimizer is not None:
        loss.backward()
        optimizer.update()
    return float(loss.data), acc


def forward_pred(model, dataset, device=None):
    loss = 0.
    acc = 0.
    iterator = chainer.iterators.SerialIterator(dataset, batch_size=4,
                                                repeat=False, shuffle=False)
    for batch in iterator:
        l, a = _run_batch(model, None, batch, device, False)
        loss += l * len(batch)
        acc += a * len(batch)
    return loss / float(len(dataset)), acc / float(len(dataset))


def train(model, optimizer, train_itr, n_epoch, dev=None, device=None,
          tmp_dir='tmp.model', lr_decay=0.995):
    loss = 0.
    acc = 0.
    min_loss = float('inf')
    min_epoch = 0
    report_tmpl = "[{:>3d}] T/loss={:0.6f} T/acc={:0.6f} D/loss={:0.6f} D/acc={:0.6f} lr={:0.6f}"
    for batch in train_itr:
        if train_itr.is_new_epoch:
            # this is not executed at first epoch
            loss_dev, acc_dev = forward_pred(model, dev, device=device)
            loss = loss / len(train_itr.dataset)
            acc = acc / len(train_itr.dataset)
            logging.info(report_tmpl.format(
                train_itr.epoch - 1, loss, acc, loss_dev, acc_dev, optimizer.alpha))
            if loss_dev < min_loss:
                min_loss = loss_dev
                min_epoch = train_itr.epoch - 1
                chainer.serializers.save_npz(tmp_dir, model)

            loss = 0.
            acc = 0.
            optimizer.alpha *= lr_decay
        if train_itr.epoch == n_epoch:
            break
        l, a = _run_batch(model, optimizer, batch, device, True)
        loss += l * len(batch)
        acc += a * len(batch)

    logging.info('loading early stopped-model at epoch {}'.format(min_epoch))
    chainer.serializers.load_npz(tmp_dir, model)

In [4]:
import argparse
import logging

import chainer

logging.basicConfig(level=logging.INFO)

def run(args):
    logging.info("Loading dataset")

    trains = create_dataset(args.train)
    logging.info("loaded {} sets for training".format(len(trains)))

    dev = create_dataset(args.dev)
    logging.info("loaded {} sets for dev".format(len(dev)))

    test = create_dataset(args.test)
    logging.info("loaded {} sets for test".format(len(test)))

    listnet = ListNet(trains[0][0].shape[1], 200, 0.0)
    optimizer = chainer.optimizers.Adam(alpha=0.0007)
    optimizer.setup(listnet)
    optimizer.add_hook(chainer.optimizer.WeightDecay(0.0005))
    #optimizer.add_hook(chainer.optimizer.GradientClipping(5.))

    train_itr = chainer.iterators.SerialIterator(trains, batch_size=1)
    train(listnet, optimizer, train_itr, 1000, dev=dev,
                   device=None)
    loss, acc = forward_pred(listnet, test, device=None)
    logging.info("Test => loss={:0.6f} acc={:0.6f}".format(loss, acc))

if __name__ == '__main__':
    p = argparse.ArgumentParser()
    p.add_argument('--train', required=True, type=str,
                   help='SNLI train json file path')
    p.add_argument('--dev', required=True, type=str,
                   help='SNLI dev json file path')
    p.add_argument('--test', required=True, type=str,
                   help='SNLI test json file path')

    # optional
    p.add_argument('-g', '--gpu', type=int, default=None, help="GPU number")
    args = p.parse_args(["--train", "MQ2008/Fold1/train.txt",
                         "--dev", "MQ2008/Fold1/vali.txt",
                         "--test", "MQ2008/Fold1/test.txt"])

    run(args)

INFO:root:Loading dataset
INFO:root:loaded 471 sets for training
INFO:root:loaded 157 sets for dev
INFO:root:loaded 156 sets for test
INFO:root:[  0] T/loss=0.141992 T/acc=0.452671 D/loss=0.150780 D/acc=0.499399 lr=0.000700
INFO:root:[  1] T/loss=0.137121 T/acc=0.473230 D/loss=0.148087 D/acc=0.489349 lr=0.000696
INFO:root:[  2] T/loss=0.137533 T/acc=0.474426 D/loss=0.145094 D/acc=0.511252 lr=0.000693
INFO:root:[  3] T/loss=0.135741 T/acc=0.482498 D/loss=0.147472 D/acc=0.507922 lr=0.000690
INFO:root:[  4] T/loss=0.135778 T/acc=0.481642 D/loss=0.148786 D/acc=0.486047 lr=0.000686
INFO:root:[  5] T/loss=0.134414 T/acc=0.479650 D/loss=0.151921 D/acc=0.493992 lr=0.000683
INFO:root:[  6] T/loss=0.133705 T/acc=0.479255 D/loss=0.145013 D/acc=0.508025 lr=0.000679
INFO:root:[  7] T/loss=0.133526 T/acc=0.486753 D/loss=0.146726 D/acc=0.505223 lr=0.000676
INFO:root:[  8] T/loss=0.133824 T/acc=0.483559 D/loss=0.151500 D/acc=0.507638 lr=0.000672
INFO:root:[  9] T/loss=0.132119 T/acc=0.484972 D/loss=0.

INFO:root:[ 90] T/loss=0.092433 T/acc=0.561673 D/loss=0.170828 D/acc=0.489721 lr=0.000446
INFO:root:[ 91] T/loss=0.091170 T/acc=0.561004 D/loss=0.168654 D/acc=0.503411 lr=0.000444
INFO:root:[ 92] T/loss=0.090641 T/acc=0.560916 D/loss=0.167244 D/acc=0.508714 lr=0.000441
INFO:root:[ 93] T/loss=0.091196 T/acc=0.560324 D/loss=0.167431 D/acc=0.492213 lr=0.000439
INFO:root:[ 94] T/loss=0.090351 T/acc=0.563406 D/loss=0.166685 D/acc=0.493304 lr=0.000437
INFO:root:[ 95] T/loss=0.090213 T/acc=0.558045 D/loss=0.166833 D/acc=0.498793 lr=0.000435
INFO:root:[ 96] T/loss=0.090897 T/acc=0.559838 D/loss=0.169038 D/acc=0.490270 lr=0.000433
INFO:root:[ 97] T/loss=0.088821 T/acc=0.562882 D/loss=0.161109 D/acc=0.499610 lr=0.000430
INFO:root:[ 98] T/loss=0.088890 T/acc=0.562096 D/loss=0.169414 D/acc=0.500984 lr=0.000428
INFO:root:[ 99] T/loss=0.088314 T/acc=0.559778 D/loss=0.177072 D/acc=0.494061 lr=0.000426
INFO:root:[100] T/loss=0.088321 T/acc=0.563032 D/loss=0.167099 D/acc=0.512477 lr=0.000424
INFO:root:

INFO:root:[182] T/loss=0.073599 T/acc=0.584461 D/loss=0.175887 D/acc=0.490650 lr=0.000281
INFO:root:[183] T/loss=0.072880 T/acc=0.582434 D/loss=0.177341 D/acc=0.498038 lr=0.000280
INFO:root:[184] T/loss=0.073477 T/acc=0.584360 D/loss=0.177690 D/acc=0.500822 lr=0.000278
INFO:root:[185] T/loss=0.072670 T/acc=0.585974 D/loss=0.182551 D/acc=0.502849 lr=0.000277
INFO:root:[186] T/loss=0.073408 T/acc=0.585919 D/loss=0.179432 D/acc=0.505205 lr=0.000276
INFO:root:[187] T/loss=0.071993 T/acc=0.582313 D/loss=0.189394 D/acc=0.505566 lr=0.000274
INFO:root:[188] T/loss=0.073370 T/acc=0.585004 D/loss=0.181881 D/acc=0.500645 lr=0.000273
INFO:root:[189] T/loss=0.073172 T/acc=0.587224 D/loss=0.186909 D/acc=0.503247 lr=0.000271
INFO:root:[190] T/loss=0.071868 T/acc=0.586961 D/loss=0.186305 D/acc=0.490074 lr=0.000270
INFO:root:[191] T/loss=0.072566 T/acc=0.584223 D/loss=0.181789 D/acc=0.499910 lr=0.000269
INFO:root:[192] T/loss=0.072018 T/acc=0.588013 D/loss=0.175797 D/acc=0.496586 lr=0.000267
INFO:root:

INFO:root:[274] T/loss=0.065771 T/acc=0.594310 D/loss=0.189176 D/acc=0.487961 lr=0.000177
INFO:root:[275] T/loss=0.065759 T/acc=0.595863 D/loss=0.183812 D/acc=0.493561 lr=0.000176
INFO:root:[276] T/loss=0.065878 T/acc=0.599798 D/loss=0.183361 D/acc=0.499806 lr=0.000175
INFO:root:[277] T/loss=0.065533 T/acc=0.598012 D/loss=0.190637 D/acc=0.493254 lr=0.000175
INFO:root:[278] T/loss=0.064965 T/acc=0.595583 D/loss=0.181077 D/acc=0.485104 lr=0.000174
INFO:root:[279] T/loss=0.065272 T/acc=0.596282 D/loss=0.185062 D/acc=0.489605 lr=0.000173
INFO:root:[280] T/loss=0.065313 T/acc=0.598103 D/loss=0.196725 D/acc=0.486731 lr=0.000172
INFO:root:[281] T/loss=0.065166 T/acc=0.593830 D/loss=0.184754 D/acc=0.495152 lr=0.000171
INFO:root:[282] T/loss=0.065548 T/acc=0.597833 D/loss=0.186760 D/acc=0.486632 lr=0.000170
INFO:root:[283] T/loss=0.065535 T/acc=0.597711 D/loss=0.185575 D/acc=0.490777 lr=0.000169
INFO:root:[284] T/loss=0.065520 T/acc=0.597448 D/loss=0.182981 D/acc=0.494869 lr=0.000169
INFO:root:

INFO:root:[366] T/loss=0.061403 T/acc=0.605048 D/loss=0.193778 D/acc=0.488823 lr=0.000112
INFO:root:[367] T/loss=0.061697 T/acc=0.601898 D/loss=0.191273 D/acc=0.491308 lr=0.000111
INFO:root:[368] T/loss=0.061221 T/acc=0.604422 D/loss=0.187749 D/acc=0.487293 lr=0.000111
INFO:root:[369] T/loss=0.061005 T/acc=0.605518 D/loss=0.189591 D/acc=0.484083 lr=0.000110
INFO:root:[370] T/loss=0.061345 T/acc=0.603259 D/loss=0.187102 D/acc=0.483799 lr=0.000110
INFO:root:[371] T/loss=0.060866 T/acc=0.603743 D/loss=0.188904 D/acc=0.490396 lr=0.000109
INFO:root:[372] T/loss=0.061397 T/acc=0.609205 D/loss=0.183057 D/acc=0.487161 lr=0.000108
INFO:root:[373] T/loss=0.060949 T/acc=0.603920 D/loss=0.188659 D/acc=0.490738 lr=0.000108
INFO:root:[374] T/loss=0.061078 T/acc=0.606808 D/loss=0.193453 D/acc=0.488481 lr=0.000107
INFO:root:[375] T/loss=0.060831 T/acc=0.603080 D/loss=0.190829 D/acc=0.491957 lr=0.000107
INFO:root:[376] T/loss=0.060880 T/acc=0.604855 D/loss=0.191189 D/acc=0.490868 lr=0.000106
INFO:root:

...

INFO:root:[550] T/loss=0.056953 T/acc=0.612553 D/loss=0.193849 D/acc=0.486959 lr=0.000044
INFO:root:[551] T/loss=0.056841 T/acc=0.614291 D/loss=0.193058 D/acc=0.482326 lr=0.000044
INFO:root:[552] T/loss=0.056714 T/acc=0.611024 D/loss=0.191977 D/acc=0.483863 lr=0.000044
INFO:root:[553] T/loss=0.056721 T/acc=0.612431 D/loss=0.196134 D/acc=0.480908 lr=0.000044
INFO:root:[554] T/loss=0.056897 T/acc=0.615781 D/loss=0.191787 D/acc=0.483944 lr=0.000044
INFO:root:[555] T/loss=0.056781 T/acc=0.612736 D/loss=0.192622 D/acc=0.484866 lr=0.000043
INFO:root:[556] T/loss=0.056810 T/acc=0.613782 D/loss=0.193040 D/acc=0.482081 lr=0.000043
INFO:root:[557] T/loss=0.056628 T/acc=0.610415 D/loss=0.192084 D/acc=0.483437 lr=0.000043
INFO:root:[558] T/loss=0.056709 T/acc=0.614101 D/loss=0.191295 D/acc=0.481092 lr=0.000043
INFO:root:[559] T/loss=0.056581 T/acc=0.610652 D/loss=0.190868 D/acc=0.486779 lr=0.000042
INFO:root:[560] T/loss=0.056717 T/acc=0.613184 D/loss=0.191480 D/acc=0.482912 lr=0.000042
INFO:root:

INFO:root:[642] T/loss=0.055286 T/acc=0.614465 D/loss=0.196079 D/acc=0.482959 lr=0.000028
INFO:root:[643] T/loss=0.056180 T/acc=0.616420 D/loss=0.192252 D/acc=0.481287 lr=0.000028
INFO:root:[644] T/loss=0.055600 T/acc=0.612867 D/loss=0.192403 D/acc=0.483715 lr=0.000028
INFO:root:[645] T/loss=0.055757 T/acc=0.616765 D/loss=0.195204 D/acc=0.480471 lr=0.000028
INFO:root:[646] T/loss=0.055461 T/acc=0.613716 D/loss=0.193885 D/acc=0.485924 lr=0.000027
INFO:root:[647] T/loss=0.055637 T/acc=0.614960 D/loss=0.193959 D/acc=0.481981 lr=0.000027
INFO:root:[648] T/loss=0.055804 T/acc=0.614950 D/loss=0.192757 D/acc=0.483939 lr=0.000027
INFO:root:[649] T/loss=0.055641 T/acc=0.611930 D/loss=0.194173 D/acc=0.481909 lr=0.000027
INFO:root:[650] T/loss=0.055575 T/acc=0.614932 D/loss=0.194068 D/acc=0.484386 lr=0.000027
INFO:root:[651] T/loss=0.055670 T/acc=0.616477 D/loss=0.194219 D/acc=0.485716 lr=0.000027
INFO:root:[652] T/loss=0.055541 T/acc=0.612352 D/loss=0.196372 D/acc=0.485282 lr=0.000027
INFO:root:

INFO:root:[734] T/loss=0.054954 T/acc=0.616069 D/loss=0.193481 D/acc=0.479038 lr=0.000018
INFO:root:[735] T/loss=0.054930 T/acc=0.613594 D/loss=0.193133 D/acc=0.480693 lr=0.000018
INFO:root:[736] T/loss=0.054927 T/acc=0.616208 D/loss=0.193795 D/acc=0.480650 lr=0.000017
INFO:root:[737] T/loss=0.054857 T/acc=0.615522 D/loss=0.192567 D/acc=0.479718 lr=0.000017
INFO:root:[738] T/loss=0.054964 T/acc=0.613669 D/loss=0.197012 D/acc=0.481661 lr=0.000017
INFO:root:[739] T/loss=0.054920 T/acc=0.617351 D/loss=0.193893 D/acc=0.476349 lr=0.000017
INFO:root:[740] T/loss=0.054874 T/acc=0.615719 D/loss=0.194252 D/acc=0.481570 lr=0.000017
INFO:root:[741] T/loss=0.054962 T/acc=0.613630 D/loss=0.195832 D/acc=0.478622 lr=0.000017
INFO:root:[742] T/loss=0.054515 T/acc=0.616608 D/loss=0.193836 D/acc=0.485911 lr=0.000017
INFO:root:[743] T/loss=0.055301 T/acc=0.614291 D/loss=0.193318 D/acc=0.483621 lr=0.000017
INFO:root:[744] T/loss=0.054904 T/acc=0.615156 D/loss=0.194980 D/acc=0.486275 lr=0.000017
INFO:root:

INFO:root:[826] T/loss=0.054457 T/acc=0.618432 D/loss=0.195920 D/acc=0.478531 lr=0.000011
INFO:root:[827] T/loss=0.054371 T/acc=0.613222 D/loss=0.195567 D/acc=0.474415 lr=0.000011
INFO:root:[828] T/loss=0.054332 T/acc=0.616995 D/loss=0.195591 D/acc=0.477655 lr=0.000011
INFO:root:[829] T/loss=0.054480 T/acc=0.615539 D/loss=0.194303 D/acc=0.471420 lr=0.000011
INFO:root:[830] T/loss=0.054392 T/acc=0.615955 D/loss=0.194405 D/acc=0.477249 lr=0.000011
INFO:root:[831] T/loss=0.054415 T/acc=0.615926 D/loss=0.195192 D/acc=0.473334 lr=0.000011
INFO:root:[832] T/loss=0.054409 T/acc=0.615862 D/loss=0.194383 D/acc=0.478401 lr=0.000011
INFO:root:[833] T/loss=0.054403 T/acc=0.617859 D/loss=0.195997 D/acc=0.476368 lr=0.000011
INFO:root:[834] T/loss=0.054346 T/acc=0.613596 D/loss=0.194327 D/acc=0.472819 lr=0.000011
INFO:root:[835] T/loss=0.054480 T/acc=0.615844 D/loss=0.195223 D/acc=0.475917 lr=0.000011
INFO:root:[836] T/loss=0.054400 T/acc=0.618297 D/loss=0.194893 D/acc=0.473415 lr=0.000011
INFO:root:

INFO:root:[918] T/loss=0.054266 T/acc=0.616291 D/loss=0.195278 D/acc=0.471603 lr=0.000007
INFO:root:[919] T/loss=0.054040 T/acc=0.614527 D/loss=0.195296 D/acc=0.471709 lr=0.000007
INFO:root:[920] T/loss=0.054106 T/acc=0.618388 D/loss=0.195667 D/acc=0.472689 lr=0.000007
INFO:root:[921] T/loss=0.053908 T/acc=0.614554 D/loss=0.195420 D/acc=0.471708 lr=0.000007
INFO:root:[922] T/loss=0.054183 T/acc=0.617774 D/loss=0.195241 D/acc=0.471516 lr=0.000007
INFO:root:[923] T/loss=0.054038 T/acc=0.614327 D/loss=0.196493 D/acc=0.471340 lr=0.000007
INFO:root:[924] T/loss=0.053791 T/acc=0.616269 D/loss=0.195047 D/acc=0.471515 lr=0.000007
INFO:root:[925] T/loss=0.053908 T/acc=0.617222 D/loss=0.196207 D/acc=0.472034 lr=0.000007
INFO:root:[926] T/loss=0.054198 T/acc=0.616204 D/loss=0.195698 D/acc=0.471650 lr=0.000007
INFO:root:[927] T/loss=0.053916 T/acc=0.614834 D/loss=0.195037 D/acc=0.472567 lr=0.000007
INFO:root:[928] T/loss=0.053996 T/acc=0.616334 D/loss=0.195504 D/acc=0.470819 lr=0.000007
INFO:root: